In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("MLib practice").getOrCreate()

In [5]:
df = spark.read.csv("test1.csv", header = True, inferSchema = True)
df.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [6]:
#Creates a new column that is dependent to age and experience column
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols = ["age", "Experience"], outputCol = "Independent Feature")

In [7]:
#Adds the new column into a new variable called output
output = assembler.transform(df)
output.show()

+---------+---+----------+------+-------------------+
|     Name|age|Experience|Salary|Independent Feature|
+---------+---+----------+------+-------------------+
|    Krish| 31|        10| 30000|        [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|         [30.0,8.0]|
|    Sunny| 29|         4| 20000|         [29.0,4.0]|
|     Paul| 24|         3| 20000|         [24.0,3.0]|
|   Harsha| 21|         1| 15000|         [21.0,1.0]|
|  Shubham| 23|         2| 18000|         [23.0,2.0]|
+---------+---+----------+------+-------------------+



In [24]:
#Created a variable for specific columns only
finalized_data = output.select("Independent Feature", "Salary")
finalized_data.show()

+-------------------+------+
|Independent Feature|Salary|
+-------------------+------+
|        [31.0,10.0]| 30000|
|         [30.0,8.0]| 25000|
|         [29.0,4.0]| 20000|
|         [24.0,3.0]| 20000|
|         [21.0,1.0]| 15000|
|         [23.0,2.0]| 18000|
+-------------------+------+



In [25]:
from pyspark.ml.regression import LinearRegression

#Train test split

#train data will have 75% while test_data will have 25% of the data
train_data, test_data = finalized_data.randomSplit([0.75,0.25])

#Needs to input the correct columns (input and output columns)
regressor = LinearRegression(featuresCol = "Independent Feature", labelCol = "Salary")
regressor = regressor.fit(train_data)

In [27]:
#Coefficients
regressor.coefficients

DenseVector([-714.2857, 3571.4286])

In [28]:
#Intercepts
regressor.intercept

26428.57142857082

In [30]:
#Prediction
pred_result = regressor.evaluate(test_data)
pred_result.predictions.show()

+-------------------+------+-----------------+
|Independent Feature|Salary|       prediction|
+-------------------+------+-----------------+
|         [23.0,2.0]| 18000|17142.85714285713|
|         [30.0,8.0]| 25000| 33571.4285714283|
|        [31.0,10.0]| 30000| 39999.9999999996|
+-------------------+------+-----------------+



In [31]:
pred_result.meanAbsoluteError,pred_result.meanSquaredError

(6476.190476190258, 58068027.21088016)